In [35]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
mesh.GetBoundaries()

('bottom', 'right', 'top', 'left')

In [36]:
y

In [37]:
for bc in mesh.GetBoundaries():
    print(bc, "is top or right?:", bc=="right" or bc=="top")

bottom is top or right?: False
right is top or right?: True
top is top or right?: True
left is top or right?: False


In [38]:
f = 1
# (inhom) Dirichlet BC
ud = CoefficientFunction(x)

# Not we set the coefficient functions for the Neumann and Robin BC..note the y --> CoefficientFunction defined by NGSolve
# Both Neumann and Robin depend on g
g = CoefficientFunction([y if bc!="bottom" else 0 for bc in mesh.GetBoundaries()])

# Robin boundary at top and right
alpha = 5
R_alpha = CoefficientFunction([alpha if bc=="right" or "top" else 0 for bc in mesh.GetBoundaries()])

In [39]:
runtimes = {"primal": -1, "mixed": -1}

Mathematical framework formulation:

$\int_{\Omega}\nabla u \nabla v -5\int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N, \Gamma_R}yv$

For NGSolve:

*$\int_{\Omega}\nabla u \nabla v - R_{alpha} \int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N, \Gamma_R} g\ v$*

In [40]:
from time import time_ns

order = 3
FESp = H1(mesh, order=order, dirichlet="bottom")
up, vp = FESp.TnT()

# Forms
ap = BilinearForm(FESp)
ap += grad(up)*grad(vp)*dx  - R_alpha*up*vp*ds

fp = LinearForm(FESp)
fp += f*vp*dx + g*vp*ds

with TaskManager():
    ap.Assemble()
    fp.Assemble()

# Gridfunction
gfup = GridFunction(FESp, "u_primal")
gfup.Set(ud, BND)

# Need to take of the Dirichlet BC
with TaskManager():
    start_p = time_ns()
    rhs = fp.vec.CreateVector()
    rhs.data = fp.vec - ap.mat * gfup.vec
    gfup.vec.data += ap.mat.Inverse(freedofs=FESp.FreeDofs()) * rhs
    delta_p = (time_ns() - start_p) * 1e-9
    print(f"Runtime primal: {delta_p}s")
    runtimes["primal"] = round(delta_p, 3)

flux = CoefficientFunction(R_alpha * grad(gfup))

Draw (gfup)
Draw (flux, mesh, "flux_primal")
R_alpha * grad(gfup)

Runtime primal: 0.07394200000000001s


# Mixed formulation: similar to Example 4.1

We want to solve example 3.4 with a mixed finite element formulation: The idea is to define a new variable $\sigma=-\lambda \nabla u .$ Then the Poisson equation $-\operatorname{div}(\lambda \nabla u)=f$ with $u=0$ on $\partial \Omega$ can be rewritten as
$$
\begin{aligned}
-\sigma-\lambda \nabla u &=0 \quad \text { in } \Omega \\
\operatorname{div}(\sigma) &=f \quad \text { in } \Omega \\
u &=0 \quad \text { on } \partial \Omega
\end{aligned}
$$
Now let $\Sigma=H(\text { div, } \Omega)$ and $V=L^{2}(\Omega),$ then a variational formulation of ( 20 read as:
Find $(\sigma, u) \in \Sigma \times V$ such that
$$
\begin{aligned}
\int_{\Omega} \frac{-1}{\lambda} \sigma \cdot \tau  \mathrm{d} x+\int_{\Omega} \operatorname{div}(\tau) u \mathrm{d} x & \forall 0 \quad \forall \tau \in \Sigma \\
\int_{\Omega} \operatorname{div}(\sigma) v \mathrm{d} x &=\int_{\Omega} f v \quad \forall v \in V
\end{aligned}
$$
Then:

1. Derive the set of equations given by (3) from (2). For this multiply the first equation
of (2) with a smooth test function $\tau$ and the second equation with a smooth test function $v \in V$ and follow the same steps as we did for the Poisson equation in the first lecture using (1)
2. Implement this method in NGSolve for the Poisson problem given in example 3.4 to get discrete solutions $\sigma_{h}, u_{h} .$ You can use the template mixed.py where a lot of additional information is given.
3. Evaluate the flux on the boundary $\partial \Omega_{1}$ using $\operatorname{tr}^{n} \sigma_{h}$.
You should now get the exact value -0.04

In [45]:
# Setup the mixed method
order_flux= order +1
V = HDiv(mesh, order=order_flux)
Q = L2(mesh, order=order, dirichlet="bottom")

FESm = FESpace([V,Q]) # Compound space
TnTm = FESm.TnT()
#print(TnT)
(sigma, um), (tau, vm) = TnTm # Unpacking in python is DOPE
#print((sigma, um), (tau, vm))
# Above is equivalent to ...
# sigma, um = FESm.TrialFunction()
# tau, vm = FESm.TestFunction()

# Setup the special function "normal"
# n = specialcf.normal(2) # from Philip
print("mesh.dim:", mesh.dim)
normal = specialcf.normal(mesh.dim)
print(normal)


# Set up forms again
am = BilinearForm(FESm)
am += (sigma*tau - div(sigma)*vm + div(tau)*um)*dx
am.Assemble()
fm = LinearForm(FESm)
fm += f*vm*dx + ud*(tau.Trace()*normal)*ds + R_alpha*um*ds
fm.Assemble()

mesh.dim: 2
coef normal vector, real, dim=2



In [46]:
# Now on to the actual problem
gfm = GridFunction(FESm, name="gf_mixed")
gfsigma, gfum = gfm.components
# Remember g from before? :D
# It was defined as scalar valued --> "need to bring it into the vector world"
print("g:", g)
print("BND:", BND)
gfsigma.Set(g*normal, definedon=BND)
#gfum.Set(ud, BND)
print("gfsigma:", gfsigma)
print("gfum:", gfum)

with TaskManager():
    start_m = time_ns()
    # fm.vec.data -= am.mat * gfm.vec
    # gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * fm.vec
    solvers.BVP(bf=am, lf=fm, gf=gfm)
    delta_m = (time_ns() - start_m) * 1e-9
    print(f"Runtime mixed: {delta_m}s")
    runtimes["mixed"] = round(delta_m, 3)

Draw (gfsigma, mesh, "flux_mixed")
Draw (gfum, mesh, "u_mixed")
#Redraw()

g: coef unary operation ' ', real
  coef class ngfem::DomainWiseCoefficientFunction, real
    coef 0, real
    coef coordinate y, real
    coef coordinate y, real
    coef coordinate y, real

BND: VorB.BND
gfsigma: gridfunction 'gf_mixed.1' on space 'HDivHighOrderFESpace(hdivho)'
nested = 0

gfum: gridfunction 'gf_mixed.2' on space 'L2HighOrderFESpace(l2ho)'
nested = 0

Runtime mixed: 0.204881s


In [47]:
# "The" gridfunction...compound space of 2 spaces --> 2 components
gfmTest = GridFunction(FESm, name="gf_mixedTest")
gfsigmaTest, gfumTest = gfmTest.components

# Let's draw something just to see how it behaves
# gfsigmaTest.Set(CoefficientFunction( (sin(10*x), sin(10*y)) ))

gfsigma.Set(g*normal, definedon=BND)
gfumTest.Set (x)

# Draw (gfsigmaTest, mesh, "sinx_siny")
Draw (gfsigmaTest, mesh, "g*normal")
Draw (gfumTest, mesh, "x")